In [2]:
!pip install tqdm>=4.62.2
!pip install Levenshtein
!pip install sentence-transformers

zsh:1: 4.62.2 not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 2.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 4.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 616.6 kB/s eta 0:00:00a 0:00:01
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.5 MB/s eta 0:00:0000:0100:01m
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-no

In [1]:
import pandas as pd
from Levenshtein import distance
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np



/tmp/ipykernel_2214/2102947824.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/rossnorman11/.pyenv/versions/3.10.6/envs/movie_recommendation_GPT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../raw_data/mpst_full_data.csv')

In [5]:
df.head()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


In [3]:
df['plot_word_count'] = df['plot_synopsis'].apply(lambda x: len(x.split()))

In [4]:
df_500 = df[df['plot_word_count'] < 500].copy()
df_500 = df_500.drop(columns = 'plot_word_count', axis = 1)

In [5]:
df_500['tag_plot'] = df_500['plot_synopsis'] + " " + df_500['tags']

In [9]:
model = SentenceTransformer('all-MiniLM-L6-v2')
df_ = df_500.copy()
df_['tag_plot'] = df_['tag_plot'].apply(lambda x : model.encode(x))
df_index = df_.pop('title')
df_ = df_[['tag_plot']]
df_ = pd.DataFrame(np.column_stack(list(zip(*df_.values))))
df_.index = df_index
df_

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
title,,,,,,,,,,,,,,,,,,,,,
Dungeons & Dragons: The Book of Vile Darkness,-0.050191,0.171308,-0.011071,0.031128,-0.015905,-0.031009,0.032038,0.016664,0.024120,-0.005058,...,-0.054060,-0.116127,0.037026,0.015278,-0.036860,0.026114,0.054591,-0.077697,0.040902,0.020990
Little Caesar,0.004349,0.016671,-0.128623,-0.034422,-0.017498,0.050455,0.079294,0.067013,-0.021096,0.005681,...,0.041640,-0.054807,-0.020852,0.032621,0.066752,-0.005349,-0.032227,0.039703,-0.048936,-0.108932
Cassandra's Dream,-0.031535,0.068079,-0.051521,-0.029258,-0.022146,0.011981,0.019802,-0.037174,0.071357,0.004537,...,-0.055937,-0.090939,-0.096302,0.070232,0.009827,0.042309,0.006600,-0.005772,0.020566,-0.081024
The Haunted,-0.096823,0.007081,0.033493,0.136775,-0.004714,-0.011421,-0.050398,-0.088864,0.105007,-0.083026,...,0.047077,-0.080827,0.100038,0.020133,-0.052866,0.025887,0.077330,-0.002609,-0.017478,0.042627
Victoria,-0.007026,0.071364,-0.076364,0.005140,0.027067,0.127636,0.060012,0.040592,0.035122,-0.034440,...,0.040684,-0.063287,-0.074967,0.006059,-0.025546,0.050380,0.023971,-0.008307,-0.047381,-0.003047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hot Potato,-0.058953,-0.032931,-0.127416,0.046263,-0.065068,-0.022222,0.100632,-0.101592,-0.031969,0.048770,...,-0.046479,-0.093698,0.006776,0.042431,0.090799,0.052930,-0.016827,0.006115,-0.118487,-0.011561
One Night of Love,-0.059396,-0.121556,-0.019065,-0.020089,-0.143418,0.031162,0.058256,0.016226,-0.019162,-0.144606,...,0.007160,-0.020355,0.011194,0.047112,0.085196,0.047241,-0.076121,-0.000444,0.044953,-0.091697
Lucky Numbers,-0.074399,0.016632,-0.004580,-0.062739,-0.085249,0.019829,0.132874,0.056214,0.041987,-0.068564,...,0.026873,-0.137031,-0.019027,-0.003781,-0.043318,-0.032821,-0.043339,-0.063091,-0.026474,0.057520


In [10]:
from sklearn.neighbors import NearestNeighbors

In [19]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(df_)

In [12]:
def find_similar_movies(df, nbrs, title):
    distances, indices = nbrs.kneighbors([df.loc[title]])
    #print(indices)#we print df data, no longer df_
    for index in indices[0][1:]:
        print('index', index)
        print(title, '->', df.iloc[index].name)

In [33]:
find_similar_movies(df_, nbrs, "Dungeons & Dragons: The Book of Vile Darkness")

index 1706
Dungeons & Dragons: The Book of Vile Darkness -> The Redeemer: Son of Satan!
index 1033
Dungeons & Dragons: The Book of Vile Darkness -> Mørke sjeler
index 252
Dungeons & Dragons: The Book of Vile Darkness -> The Eschatrilogy: Book of the Dead
index 4427
Dungeons & Dragons: The Book of Vile Darkness -> Teenage Caveman
index 1046
Dungeons & Dragons: The Book of Vile Darkness -> Berserker


In [11]:
def input_prompt(df, prompt):
    last_ind = len(df)
    encoded_prompt = model.encode(prompt)
    new_data = np.transpose(pd.DataFrame(list(zip(encoded_prompt))))
    df = pd.concat([df,new_data])
    nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(df)
    distances, indices = nbrs.kneighbors([df.iloc[last_ind, :]])
    #print(indices)#we print df data, no longer df_
    for index in indices[0][1:]:
        print(df.iloc[index].name)

In [14]:
prompt="horror about a clown that lives in a sewer"
input_prompt(df_, prompt)

All Hallows' Eve
All Hallows' Eve 2
Clownhouse
Zelyonyy slonik
Splatterhouse


In [48]:
df_500.shape

(4771, 6)